In [2]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [3]:
readRDS = robjects.r['readRDS']
df = readRDS('../../data/wos_ngrams_machine_learning_2_2011_2015_/1_machine_learning_2_2011_2015__query_results.RDS')
df = pandas2ri.ri2py(df)

In [44]:
import os

folder = '../../data/ml'
if not os.path.exists(folder):
    os.makedirs(folder)

for i, name in enumerate(df.names):
    if type(df[i]) == robjects.vectors.DataFrame:
        df[i].to_csvfile('./%s/%s.csv' % (folder, name))

In [45]:
for pos1, name1 in enumerate(df.names):
    print('[%d] %s' % (pos1, name1))
    if not df[pos1].names:
        print('    NULL')
    else:
        for pos2, name2 in enumerate(df[pos1].names):
            print('    [%d] %s' % (pos2, name2))
    print()

[0] specificity_statistics
    [0] ngram
    [1] ngramids
    [2] local_count
    [3] local_sum
    [4] corpus_count
    [5] corpus_sum
    [6] specificity
    [7] ngram_length

[1] occurrence_by_year
    [0] ngram
    [1] pubyear
    [2] count
    [3] rel_freq

[2] cooccurrence_by_year
    [0] ngram1
    [1] ngram2
    [2] pubyear
    [3] count

[3] corpus_occurrence_by_year
    [0] pubyear
    [1] freq

[4] technology_cooccurrence
    NULL



In [6]:
import pandas as pd
source = pd.Series(list(df[2][0]))
target = pd.Series(list(df[2][1]))

In [7]:
from collections import Counter
def connections(ngram):
    return Counter(list(target[source == ngram]) + list(source[target == ngram]))

In [18]:
connections('machine learning')

Counter({'data mining': 5,
         'feature selection': 5,
         'death certificate': 1,
         'computer vision': 4,
         'graph kernel': 1,
         'approximation space': 1,
         'dangerousness forecast': 1,
         'classifier ensemble': 3,
         'direction distribution': 1,
         'boolean matrix decomposition': 1,
         'current paper': 2,
         'decision tree': 3,
         'data clustering': 2,
         'forecasting accuracy': 1,
         'endolymphatic hydrop': 1,
         'data model': 1,
         'at-risk adolescent': 1,
         'diagnosis system': 1,
         'autism spectrum disorder': 1,
         'classification version': 1,
         'criminal behavior': 1,
         'fmr image': 1,
         'image content': 1,
         'feature number': 2,
         'insufficient infrastructure': 1,
         'diagnostic error': 1,
         'immense potential': 1,
         'coreference resolution': 1,
         'lidar composite metric': 1,
         'knowledge reposi

In [19]:
ngrams = set()
for ngram in df[0][1]:
    ngrams.add(ngram)
for ngram in df[1][0]:
    ngrams.add(ngram)
for ngram in df[2][0]:
    ngrams.add(ngram)
for ngram in df[2][1]:
    ngrams.add(ngram)
ngrams = list(ngrams)
ngrams_ids = {ngram: pos for pos, ngram in enumerate(ngrams)}
len(ngrams)

39747

In [37]:
years = set()
for year in df[1][1]:
    years.add(year)
for year in df[2][2]:
    years.add(year)
for year in df[3][0]:
    years.add(year)
years = [year for year in sorted(years) if type(year) is int]
years_ids = {year: pos for pos, year in enumerate(years)}
min_year, max_year = min(years), max(years)
min_year, max_year

(1931, 2017)

In [38]:
res = {}

In [39]:
res['edge_source'] = []
res['edge_target'] = []
res['edge_date'] = []
res['edge_weight'] = []
for ngram1, ngram2, year, count in zip(df[2][0], df[2][1], df[2][2], df[2][3]):
    res['edge_source'].append(ngrams_ids[ngram1])
    res['edge_target'].append(ngrams_ids[ngram2])
    res['edge_date'].append(int(year))
    res['edge_weight'].append(int(count))

In [40]:
res['node_label'] = []
for ngram in ngrams:
    res['node_label'].append(ngram)

In [41]:
res['nodeDate_node'] = []
res['nodeDate_date'] = []
res['nodeDate_count'] = []
for ngram, year, count in zip(df[1][0], df[1][1], df[1][2]):
    res['nodeDate_node'].append(ngrams_ids[ngram])
    res['nodeDate_date'].append(year)
    res['nodeDate_count'].append(int(count))

In [46]:
import json
json.dump(res, open('../../data/ml/graph.json', 'w'))